In [1]:
# just here
%load_ext autoreload
%autoreload 2

In [2]:
# imports
from copy import copy
import numpy as np
import pandas as pd
import pynapple as nap
import matplotlib.pyplot as plt
import iblphotometry.loaders as loaders
import iblphotometry.plots as plots

In [ ]:
# KCENIA / isosbestic specific
from one.api import ONE
one = ONE(cache_dir='/mnt/h0/kb/data/one')
data_loader = loaders.KceniaLoader(one)
df = pd.read_csv('/home/georg/code/ibl-photometry/src/local/website.csv')
eids = list(df['eid'])[:20]  # <- debug

In [ ]:
pid, pname = data_loader.eid2pid(eids[14])
raw_photometry = data_loader.load_photometry_data(pid=pid[0])
plots.plot_isosbestic_overview(raw_photometry['raw_calcium'], raw_photometry['raw_isosbestic'])

In [ ]:
from iblphotometry.outlier_detection import remove_spikes
from iblphotometry.bleach_corrections import lowpass_bleachcorrect
from iblphotometry.bleach_corrections import isosbestic_correct
from iblphotometry.sliding_operations import sliding_mad
from iblphotometry.pipelines import run_pipeline

pipeline = [
      (remove_spikes, dict(sd=5)),
      # (lowpass_bleachcorrect, dict(filter_params=dict(N=3, Wn=0.01, btype='lowpass'))),
      (isosbestic_correct, dict(on_columns=False,
                                signal_name='raw_calcium',
                                reference_name='raw_isosbestic',
                                correction_method='subtract-divide',
                              #   regression_method='huber',
                                lowpass_isosbestic=dict(N=3, Wn=0.01, btype='lowpass')),),
      # (sliding_mad, dict(w_len=120,overlap=90)),

]

# run pipeline
photometry = run_pipeline(raw_photometry, pipeline)
plots.plot_Tsd(photometry)

In [ ]:
# from iblphotometry.bleach_corrections import IsosbesticCorrection
# iso = IsosbesticCorrection()
# tsd = iso.correct(raw_photometry['raw_isosbestic'], raw_photometry['raw_calcium'])
raw_photometry = data_loader.load_photometry_data(pid=pid[0])
# plt.plot(raw_photometry['raw_isosbestic'])
raw_photometry = data_loader.load_photometry_data(pid=pid[0])
plt.plot(raw_photometry['raw_isosbestic'].d, raw_photometry['raw_calcium'].d,'.')